In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [ ]:
!pwd

In [4]:
import time
import random
from typing import List
import spacy
import openai
import numpy as np
import wandb
from datasets import load_dataset
from mega.data.load_datasets import load_xnli_dataset
from mega.data.data_utils import choose_few_shot_examples
from mega.prompting.instructions import INSTRUCTIONS
from mega.prompting.prompting_utils import load_prompt_template
from mega.utils.env_utils import load_openai_env_variables
# from mega.models.completion_models import get_model_pred, gpt3x_completion
from mega.models.tag_models import gpt3x_tagger, palm_tagger
from mega.models.completion_models import gpt3x_completion
from mega.prompting.prompting_utils import construct_prompt, construct_qa_prompt, construct_tagging_prompt
from mega.data.load_datasets import load_tagging_dataset
from seqeval.metrics import f1_score
from tqdm.notebook import tqdm
from evaluate import load

# Set seed
random.seed(42)
np.random.seed(42)

In [5]:
# Make sure that {env_name}.env file is present in the envs/ directory
from dotenv import load_dotenv
env_name = "melange"
# load_env(env_name=env_name)
load_openai_env_variables()

In [32]:
openai.api_version = "2023-03-15-preview"
openai.api_version

'2023-03-15-preview'

In [33]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
openai.api_key

'470ba786e34642c9b765366a5b69c983'

In [6]:
model = "gpt-35-turbo"
pivot_lang = "en"
tgt_lang = "en"
prompt_name = "structure_prompting_chat"
few_shot_k = 8
dataset = "udpos"
# short_contexts = False
max_tokens = 100

In [7]:
config = {
    "model" : model,
    "pivot_lang": pivot_lang,
    "tgt_lang": tgt_lang,
    "prompt_name": prompt_name,
    "few_shot_k": few_shot_k,
    "dataset": dataset,
    "max_tokens": max_tokens
}

# wandb.init(project="GPT-4-eval", entity="scai-msri", config=config)

In [6]:
class SpacySentenceTokenizer:
    
    def __init__(self):
        self.nlp = spacy.load('xx_ent_wiki_sm')
        self.nlp.add_pipe("sentencizer")
        
    def __call__(self, text: str) -> List[str]:
        return list(map(lambda span: span.text, self.nlp(text).sents))


In [ ]:
# def load_tagging_dataset(
#     dataset: str,
#     lang: str,
#     split: str,
#     dataset_frac: float = 1.0,
#     xtreme_dir: str = "xtreme/download",
#     delimiter: str = "_",
# ):

#     split = "dev" if split == "validation" else split

#     filename = f"{xtreme_dir}/{dataset}/{split}-{lang}.tsv"
#     inputs, labels = read_conll_data(filename)

#     dataset = Dataset.from_dict({"tokens": inputs, "tags": labels})
#     dataset = dataset.map(
#         lambda example: {
#             "tagged_tokens": [f"{token}{delimiter}{tag}"
#             for token, tag in zip(example["tokens"], example["tags"])]
#         }
#     )

#     N = len(dataset)
#     selector = np.arange(int(N * dataset_frac))
#     return dataset.select(selector)


In [8]:
train_dataset = load_tagging_dataset(dataset,
                                lang = pivot_lang,
                                split="dev")
test_dataset = load_tagging_dataset(dataset,
                                lang = tgt_lang,
                                split="test")

Map:   0%|          | 0/3973 [00:00<?, ? examples/s]

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

In [9]:
test_dataset[0]

{'tokens': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?'],
 'tags': ['PRON', 'SCONJ', 'PROPN', 'VERB', 'ADP', 'PROPN', 'PUNCT'],
 'tagged_tokens': ['What_PRON',
  'if_SCONJ',
  'Google_PROPN',
  'Morphed_VERB',
  'Into_ADP',
  'GoogleOS_PROPN',
  '?_PUNCT']}

In [10]:
train_examples = choose_few_shot_examples(
        train_dataset, few_shot_k, selection_criteria="random")

In [11]:
PROMPTS_DICT = {
    "structure_prompting": """C: {context}\nT: {tagged}""",
    "structure_prompting_chat": """Tag the following sentence: "{context}"\n{tagged}"""
}

In [12]:
prompt_template = PROMPTS_DICT[prompt_name]

In [13]:
prompt_template

'Tag the following sentence: "{context}"\n{tagged}'

In [18]:
# Loading instruction for the task
instruction = INSTRUCTIONS[dataset]
print(instruction)

You are an NLP assistant whose purpose is to perform Part of Speech (PoS) Tagging. PoS tagging is the process of marking up a word in a text (corpus) as corresponding to a particular part of speech, based on both its definition and its context. You will need to use the tags defined below:
    1. ADJ: adjective
    2. ADP: adposition
    3. ADV: adverb
    4. AUX: auxiliary
    5. CCONJ: coordinating-conjunction
    6. DET: determiner
    7. INTJ: interjection
    8. NOUN: noun
    9. NUM: numeral
    10. PART: particle
    11. PRON: pronoun
    12. PROPN: proper-noun
    13. PUNCT: punctuation
    14. SCONJ: subordinating-conjunction
    15. SYM: symbol
    16. VERB: verb
    17. X: other
    Do not try to answer the question! Just tag each token in the sentence.


In [60]:
train_examples

[{'tokens': ['In',
   '1599',
   ',',
   'a',
   'partnership',
   'of',
   'company',
   'members',
   'built',
   'their',
   'own',
   'theatre',
   'on',
   'the',
   'south',
   'bank',
   'of',
   'the',
   'River',
   'Thames',
   ',',
   'which',
   'they',
   'called',
   'the',
   'Globe',
   '.'],
  'tags': ['ADP',
   'NUM',
   'PUNCT',
   'DET',
   'NOUN',
   'ADP',
   'NOUN',
   'NOUN',
   'VERB',
   'DET',
   'ADJ',
   'NOUN',
   'ADP',
   'DET',
   'ADJ',
   'NOUN',
   'ADP',
   'DET',
   'PROPN',
   'PROPN',
   'PUNCT',
   'PRON',
   'PRON',
   'VERB',
   'DET',
   'NOUN',
   'PUNCT'],
  'tagged_tokens': ['In_ADP',
   '1599_NUM',
   ',_PUNCT',
   'a_DET',
   'partnership_NOUN',
   'of_ADP',
   'company_NOUN',
   'members_NOUN',
   'built_VERB',
   'their_DET',
   'own_ADJ',
   'theatre_NOUN',
   'on_ADP',
   'the_DET',
   'south_ADJ',
   'bank_NOUN',
   'of_ADP',
   'the_DET',
   'River_PROPN',
   'Thames_PROPN',
   ',_PUNCT',
   'which_PRON',
   'they_PRON',
   'called

In [14]:
valid_labels = set()
for example in train_examples:
    valid_labels.update(example["tags"])
valid_labels = list(valid_labels)
valid_labels

['PROPN',
 'PRON',
 'PUNCT',
 'VERB',
 'NUM',
 'X',
 'CCONJ',
 'ADP',
 'DET',
 'SCONJ',
 'ADV',
 'AUX',
 'ADJ',
 'SYM',
 'NOUN']

In [14]:
test_example = test_dataset[186]

prompt, label = construct_tagging_prompt(
    train_examples,
    test_example,
    prompt_template=prompt_template,
    chat_prompt=False,
    instruction="Do not try to answer the question. Just tag each token in the sentence."
)
print(prompt)

Tag the following sentence: "In 1599 , a partnership of company members built their own theatre on the south bank of the River Thames , which they called the Globe ."
In_ADP 1599_NUM ,_PUNCT a_DET partnership_NOUN of_ADP company_NOUN members_NOUN built_VERB their_DET own_ADJ theatre_NOUN on_ADP the_DET south_ADJ bank_NOUN of_ADP the_DET River_PROPN Thames_PROPN ,_PUNCT which_PRON they_PRON called_VERB the_DET Globe_NOUN ._PUNCT

Tag the following sentence: "There he is , built like King Kong , as ambitious as the Empire State Building , as wide-eyed as Fay Wray , and as much a dream , an invention , as the movies and America itself ."
There_ADV he_PRON is_VERB ,_PUNCT built_VERB like_ADP King_PROPN Kong_PROPN ,_PUNCT as_ADV ambitious_ADJ as_ADP the_DET Empire_NOUN State_NOUN Building_NOUN ,_PUNCT as_ADV wide-eyed_ADJ as_ADP Fay_PROPN Wray_PROPN ,_PUNCT and_CCONJ as_ADV much_ADV a_DET dream_NOUN ,_PUNCT an_DET invention_NOUN ,_PUNCT as_SCONJ the_DET movies_NOUN and_CCONJ America_PROPN i

In [15]:
test_example["tokens"]

['Even',
 'after',
 'the',
 'attacks',
 'on',
 'September',
 '11',
 ',',
 'Bush',
 'was',
 'obsessing',
 'about',
 'Iraq',
 '.']

In [16]:
preds = gpt3x_completion(
    prompt,
    model,
    test_example["tokens"],
    temperature=0,
    max_tokens=100
)

KeyboardInterrupt: 

In [20]:
preds

[autoreload of mega.models.tag_models failed: Traceback (most recent call last):
  File "/anaconda/envs/mega/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/anaconda/envs/mega/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/anaconda/envs/mega/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/t-sahuja/MultilingualBlanketEval/mega/models/tag_models.py", line 227, in <module>
    @backoff.on_exception(backoff.expo, ResourceExhausted, max_time=300)
NameError: name 'ResourceExhausted' is not defined
]


['PRON', 'AUX', '?', 'PRON', '?', '?', '?', '?', '?']

In [19]:
" ".join(test_example["tagged_tokens"])

"Donner_VERB le_DET nom_NOUN d'_ADP une_DET équipe_NOUN pour_ADP laquelle_PRON Tim_PROPN Crews_PROPN a_AUX joué_VERB ._PUNCT"

In [41]:
prompt_2 = INSTRUCTIONS['udpos'] + prompt

TypeError: can only concatenate str (not "list") to str

In [19]:
test_example = test_dataset[0]
prompt, label = construct_tagging_prompt(
    train_examples,
    test_example,
    prompt_template=prompt_template,
    chat_prompt=False,
    instruction=instruction
)
print(prompt)

Tag the following sentence: "In 1599 , a partnership of company members built their own theatre on the south bank of the River Thames , which they called the Globe ."
In_ADP 1599_NUM ,_PUNCT a_DET partnership_NOUN of_ADP company_NOUN members_NOUN built_VERB their_DET own_ADJ theatre_NOUN on_ADP the_DET south_ADJ bank_NOUN of_ADP the_DET River_PROPN Thames_PROPN ,_PUNCT which_PRON they_PRON called_VERB the_DET Globe_NOUN ._PUNCT

Tag the following sentence: "There he is , built like King Kong , as ambitious as the Empire State Building , as wide-eyed as Fay Wray , and as much a dream , an invention , as the movies and America itself ."
There_ADV he_PRON is_VERB ,_PUNCT built_VERB like_ADP King_PROPN Kong_PROPN ,_PUNCT as_ADV ambitious_ADJ as_ADP the_DET Empire_NOUN State_NOUN Building_NOUN ,_PUNCT as_ADV wide-eyed_ADJ as_ADP Fay_PROPN Wray_PROPN ,_PUNCT and_CCONJ as_ADV much_ADV a_DET dream_NOUN ,_PUNCT an_DET invention_NOUN ,_PUNCT as_SCONJ the_DET movies_NOUN and_CCONJ America_PROPN i

In [20]:
preds = gpt3x_tagger(
    prompt,
    model,
    test_example["tokens"],
    one_shot_tag=False,
    temperature=0,
    max_tokens=5
)

Tag the following sentence: "In 1599 , a partnership of company members built their own theatre on the south bank of the River Thames , which they called the Globe ."
In_ADP 1599_NUM ,_PUNCT a_DET partnership_NOUN of_ADP company_NOUN members_NOUN built_VERB their_DET own_ADJ theatre_NOUN on_ADP the_DET south_ADJ bank_NOUN of_ADP the_DET River_PROPN Thames_PROPN ,_PUNCT which_PRON they_PRON called_VERB the_DET Globe_NOUN ._PUNCT

Tag the following sentence: "There he is , built like King Kong , as ambitious as the Empire State Building , as wide-eyed as Fay Wray , and as much a dream , an invention , as the movies and America itself ."
There_ADV he_PRON is_VERB ,_PUNCT built_VERB like_ADP King_PROPN Kong_PROPN ,_PUNCT as_ADV ambitious_ADJ as_ADP the_DET Empire_NOUN State_NOUN Building_NOUN ,_PUNCT as_ADV wide-eyed_ADJ as_ADP Fay_PROPN Wray_PROPN ,_PUNCT and_CCONJ as_ADV much_ADV a_DET dream_NOUN ,_PUNCT an_DET invention_NOUN ,_PUNCT as_SCONJ the_DET movies_NOUN and_CCONJ America_PROPN i

In [67]:
print(test_example["tokens"])

['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']


In [66]:
print(preds)

['PRON', 'SCONJ', 'PROPN', 'VERB', 'Into', '?_PUNCT', 'PUNCT']


In [ ]:
print(prompt)

In [36]:
preds

['PRON', 'AUX', '?', 'PRON', '?', '?', '?', '?', '?']

In [26]:
!export GOOGLE_APPLICATION_CREDENTIALS="/home/t-sahuja/MultilingualBlanketEval/envs/vertex_api.json"

In [25]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/t-sahuja/MultilingualBlanketEval/envs/vertex_api.json"

'Tag the following sentence: "James Graham était le chef du clan Graham ."\nJames_PROPN Graham_PROPN était_AUX le_DET chef_NOUN du_ADP clan_NOUN Graham_PROPN ._PUNCT\n\nTag the following sentence: "C\' est là qu\' à 26 ans , il devient le plus jeune chef récompensé par deux étoiles ."\nC\'_PRON est_AUX là_ADV qu\'_SCONJ à_ADP 26_NUM ans_NOUN ,_PUNCT il_PRON devient_VERB le_DET plus_ADV jeune_ADJ chef_NOUN récompensé_VERB par_ADP deux_NUM étoiles_NOUN ._PUNCT\n\nTag the following sentence: "La formation de craquelures est appelée craquelage ( crazing en anglais ) ."\nLa_DET formation_NOUN de_ADP craquelures_NOUN est_AUX appelée_VERB craquelage_NOUN (_PUNCT crazing_NOUN en_ADP anglais_NOUN )_PUNCT ._PUNCT\n\nTag the following sentence: "l\' incidence des fractures en % ( IC )"\nl\'_DET incidence_NOUN des_ADP fractures_NOUN en_ADP %_NOUN (_PUNCT IC_PROPN )_PUNCT\n\nTag the following sentence: "Pour remédier à cela , les Chinois conduisaient d\' abord le gaz dans un grand réservoir en bois

In [34]:
from mega.models.tag_models import gpt3x_tagger, palm_tagger


In [35]:
pred_palm = palm_tagger(
    prompt,
    "text-bison-32k",
    lang=pivot_lang,
    test_tokens=test_example["tokens"],
    max_tokens=5,
    temperature=0.,
)

In [36]:
pred_palm

['PRON', 'CCONJ', 'PROPN', 'VERB', 'ADP', 'X_NOUN', 'PUNCT']

In [32]:
test_example

{'tokens': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?'],
 'tags': ['PRON', 'SCONJ', 'PROPN', 'VERB', 'ADP', 'PROPN', 'PUNCT'],
 'tagged_tokens': ['What_PRON',
  'if_SCONJ',
  'Google_PROPN',
  'Morphed_VERB',
  'Into_ADP',
  'GoogleOS_PROPN',
  '?_PUNCT']}

In [ ]:
test_example["tags"]

In [ ]:
get_model_pred(
    train_examples,
    test_example,
    prompt_template,
    verbalizer={},
    model=model,
    chat_prompt=True,
    instruction=instruction,
    one_shot_tag=True,
    max_tokens=max_tokens
    
)

In [ ]:
preds = [pred if pred != "" else np.random.choice(valid_labels) for pred in preds]

In [ ]:
print(preds)

In [ ]:
print(f"Prediction: {preds}")
print(f"Label: {label}")

f1_score([preds], [label])

# prediction = {"prediction_text": pred, "id": test_example["id"]}
# reference = {}
# reference["answers"] = test_example["answers"]
# reference["id"] = test_example["id"]
# results = squad_metric.compute(
#             predictions=[prediction],
#             references=[reference]
#         )

In [ ]:
f1_sum = 0
em_sum = 0
avg_em = 0
avg_f1 = 0

run_details = {"num_calls": 0}

pbar = tqdm(enumerate(test_dataset.select(range(1000))))

for i, test_example in pbar:    
    prompt, label = construct_tagging_prompt(
        train_examples,
        test_example,
        prompt_template=prompt_template,
        chat_prompt=True,
        instruction=instruction
    )
    preds = gpt3x_tagger(
        prompt,
        model,
        test_example["tokens"],
        one_shot_tag=True,
        temperature=0,
        max_tokens=100
    )
    preds = [pred if pred != "" else np.random.choice(valid_labels) for pred in preds]
    f1_sum += f1_score([preds], [label])
        
    avg_f1 = f1_sum / (i+1)
    
#     wandb.log({"f1": avg_f1})
#     wandb.log(run_details)
    pbar.set_description(f"f1: {avg_f1}")
#     time.sleep(1/2)